# Final Project Data Analysis
### Data Cleaning
For the final project a significant amount of data had to be analyzed. The data we will be analyzing was obtained from the console log of the web application and placed into a text file. For ease of integer parsing, the X, Y and time step of each value are not correlated and are simply listed in order. Correlating them together is part of the initial cleaning of data.

In [1]:
# Initialize packages to be used, enable matplotlib inline
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import csv

In [86]:
# First we start by turning our text file lines into a clean list
f = open("data/base_trial.csv")
empty_list = []
for row in f:
    i = int(row.strip())
    empty_list.append(i)

# Now let's do a test print for verification
print(empty_list[0: 15])

[166, 19, 13217713, 161, 25, 13217729, 154, 38, 13217755, 147, 54, 13217771, 128, 95, 13217793]


In [87]:
# Now that we have our initial list, we need to organize the data
# into individual correlated arrays
complete_list = [empty_list[x:x+3] for x in range(0, len(empty_list),3)]

# Now let's do another small test print for verification
print(complete_list[0:15])

[[166, 19, 13217713], [161, 25, 13217729], [154, 38, 13217755], [147, 54, 13217771], [128, 95, 13217793], [110, 132, 13217814], [107, 143, 13217837], [107, 152, 13217845], [113, 189, 13217880], [137, 221, 13217903], [149, 232, 13217925], [173, 248, 13217947], [195, 262, 13217955], [225, 278, 13217991], [234, 282, 13218012]]


In [77]:
# Very nice! Now that we have the data in correlated arrays, lets combine it
# all together in a function so that we don't need to re-do this every time.
def dataClean(file):
    f = open(file)
    empty_list = []
    for row in f:
        i = int(row.strip())
        empty_list.append(i)
    complete_list = [empty_list[x:x+3] for x in range(0, len(empty_list),3)]
    return complete_list

In [89]:
# Now that the data has been reduced into a function, clean data from the
# console output can be produced on the fly!
print(dataClean("data/base_trial.csv")[1:9])

[[161, 25, 13217729], [154, 38, 13217755], [147, 54, 13217771], [128, 95, 13217793], [110, 132, 13217814], [107, 143, 13217837], [107, 152, 13217845], [113, 189, 13217880]]


In [91]:
# The function can be utilized for both .csv and .txt files without need
# to change anything.
print(dataClean("data/base_trial.txt")[1:9])

[[161, 25, 13217729], [154, 38, 13217755], [147, 54, 13217771], [128, 95, 13217793], [110, 132, 13217814], [107, 143, 13217837], [107, 152, 13217845], [113, 189, 13217880]]


### Data Plotting
Now that we have clean/useful data, we can proceed to plot it.